<a href="https://colab.research.google.com/github/michalis0/BigScaleAnalytics/blob/master/week2/solutions/Bsa_lab_sql_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BSA Lab Week 2 - SQL in Python

There is an iPython extension that allows us to use SQL from a notebook by means of so-called "magic" commands (%...). You can choose from several SQL engines (PostgreSQL, MySQL, etc.). For these exercises, we will use SQLite. Rather than a full-fledged client-server database engine, SQLite can be embedded onto any program.

In [ ]:
%load_ext sql
%sql sqlite://

import pandas as pd

## <font color = 'green'>World Cup Data</font>

This dataset consists of two CSV files, `Players.csv` and `Teams.csv`, which have already been joined into a third one for your convenience (`PlayersExt.csv`). We are loading them directly from the GitHub repository, and then persisting the tables to our SQL database so that we can run SQL queries (as opposed to using pandas, for example).

In [ ]:
# Load Players
players_url = "https://raw.githubusercontent.com/michalis0/BigScaleAnalytics/master/week2/data/Players.csv"
Players = pd.read_csv(players_url, index_col=0, encoding="utf-8")
%sql drop table if exists Players;
%sql persist Players

# Load Teams
teams_url = "https://raw.githubusercontent.com/michalis0/BigScaleAnalytics/master/week2/data/Teams.csv"
Teams = pd.read_csv(teams_url, index_col=0, encoding="utf-8")
%sql drop table if exists Teams;
%sql persist Teams

# Load joined tables
playersext_url = "https://raw.githubusercontent.com/michalis0/BigScaleAnalytics/master/week2/data/PlayersExt.csv"
PlayersExt = pd.read_csv(playersext_url, index_col=0, encoding="utf-8")
%sql drop table if exists PlayersExt;
%sql persist PlayersExt

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


'Persisted playersext'

### Basic Queries Quick Recap

Let's run our first SQL queries in order to see what the table attributes and values look like. Here are a couple of things to note about the queries:

* As you can see, any text following the `%%sql` command is interpreted as query language.
* For the SQL keywords (SELECT, FROM, GROUP BY, etc.), it doesn't matter whether you use lowercase or uppercase.
* When you need to use quotes, both single quotes (' ') and double quotes (" ") can be used.
* You can also spread your statements across several lines for better readability.

In [ ]:
%%sql
SELECT * FROM Players LIMIT 5

 * sqlite://
Done.


surname,team,position,minutes,shots,passes,tackles,saves
Abdoun,Algeria,midfielder,16,0,6,0,0
Belhadj,Algeria,defender,270,1,146,8,0
Boudebouz,Algeria,midfielder,74,3,28,1,0
Bougherra,Algeria,defender,270,1,89,11,0
Chaouchi,Algeria,goalkeeper,90,0,17,0,2


In [ ]:
%%sql
select * from Teams limit 5

 * sqlite://
Done.


team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
Brazil,1,5,3,1,1,9,4,7,2
Spain,2,6,5,0,1,7,2,3,0
Portugal,3,4,1,2,1,7,1,8,1
Netherlands,4,6,6,0,0,12,5,15,0
Italy,5,3,0,2,1,4,5,5,0


In [ ]:
%%sql
select *
from PlayersExt
limit 5

 * sqlite://
Done.


surname,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards,position,minutes,shots,passes,tackles,saves
Abdoun,Algeria,30,3,0,1,2,0,2,4,2,midfielder,16,0,6,0,0
Belhadj,Algeria,30,3,0,1,2,0,2,4,2,defender,270,1,146,8,0
Boudebouz,Algeria,30,3,0,1,2,0,2,4,2,midfielder,74,3,28,1,0
Bougherra,Algeria,30,3,0,1,2,0,2,4,2,defender,270,1,89,11,0
Chaouchi,Algeria,30,3,0,1,2,0,2,4,2,goalkeeper,90,0,17,0,2


Simple query on one table.   
Use WHERE for adding one or several conditions.   
Use LIKE "%er%" to search for words that include the fragment "er".   
Use ORDER BY to sort.

In [ ]:
%%sql
SELECT surname, team, minutes, passes
FROM Players
WHERE team="Switzerland" AND surname LIKE "%er%"
ORDER BY surname

 * sqlite://
Done.


surname,team,minutes,passes
Derdiyok,Switzerland,191,47
Fernandes,Switzerland,212,56
Inler,Switzerland,270,138
Lichtsteiner,Switzerland,270,80
Senderos,Switzerland,36,9
Ziegler,Switzerland,270,71
von Bergen,Switzerland,234,79


Simple aggregation question - using GROUP BY - aggregare results by some criteria.   
Use AS to set aliases.   
Use WHERE as a condition before GROUP BY and HAVING as a condition after GROUP BY.

In [ ]:
%%sql
SELECT team, COUNT(*) AS number_players_listed
FROM Players
GROUP BY team
HAVING number_players_listed<20
ORDER BY number_players_listed DESC
LIMIT 5

 * sqlite://
Done.


team,number_players_listed
Australia,19
Brazil,19
Denmark,19
England,19
France,19


Simple join question - using JOIN (default INNER) - when we need information from several tables in the database and we have a common key.

In [ ]:
%%sql
SELECT surname, position, minutes, T.team, ranking, games, wins, draws, losses
FROM Players as P
JOIN Teams AS T
ON P.team=T.team
WHERE surname <"C"
LIMIT 5

 * sqlite://
Done.


surname,position,minutes,team,ranking,games,wins,draws,losses
Abdoun,midfielder,16,Algeria,30,3,0,1,2
Abe,midfielder,351,Japan,45,4,2,1,1
Abidal,defender,180,France,9,3,0,1,2
Abou Diaby,midfielder,270,France,9,3,0,1,2
Aboubakar,forward,46,Cameroon,19,3,0,0,3


### Basic Exercises

The first three questions are already solved for you, so that you have concrete examples of queries. Try to solve the remaining ones!

*1)  Which player on a team ending with "ia" played less than 200 minutes and made more than 100 passes? Return the player's surname and team.*

**Hint**: To check if attribute A contains a (sub)string S, use the LIKE keyword (e.g. `A like '%S%'`). The % sign indicates a wildcard.

In [ ]:
%%sql
select surname, team
from Players
where Players.team like '%ia'
  and Players.minutes < 200 
  and Players.passes > 100

 * sqlite://
Done.


surname,team
Kuzmanovic,Serbia


*2) Find all players who made more than 20 shots. Return all player information in descending order of shots made.*

**Hint**: Sorting results is done via the ORDER BY keyword. The default order is ascending (ASC). If you want descending order, use DESC (e.g. `ORDER BY column_1, column_2 DESC`).

In [ ]:
%%sql
select *
from Players
where shots > 20
order by shots desc

 * sqlite://
Done.


surname,team,position,minutes,shots,passes,tackles,saves
Gyan,Ghana,forward,501,27,151,1,0
Villa,Spain,forward,529,22,169,2,0
Messi,Argentina,forward,450,21,321,10,0


*3) Which team has the highest average number of passes per minute played? Return the team's name and average number of passes per minute.*

**Hint #1**: You can compute a team's average number of passes per minute played by dividing the total number of passes by the total number of minutes. To force floating point division, multiply one operand by 1.0.

**Hint #2**: Consider using the LIMIT keyword.

In [ ]:
%%sql
SELECT *, (1.0 * total_passes / total_minutes) AS avg_ppm
FROM (
    SELECT team, sum(passes) AS total_passes, sum(minutes) AS total_minutes
    FROM PlayersExt
    GROUP BY team)
ORDER BY avg_ppm DESC
LIMIT 1

 * sqlite://
Done.


team,total_passes,total_minutes,avg_ppm
Spain,3701,5940,0.6230639730639731


*4) Find the goalkeepers of teams that played more than four games. List the surname of the goalkeeper, the team, and the number of minutes the goalkeeper played.*

**Hint**: Use the `PlayersExt` table.

In [ ]:
%%sql
select surname, team, minutes
from PlayersExt
where position like "goal%"
  and games > 4

 * sqlite://
Done.


surname,team,minutes
Romero,Argentina,450
Julio Cesar,Brazil,450
Neuer,Germany,540
Kingson,Ghana,510
Stekelenburg,Netherlands,540
Villar,Paraguay,480
Casillas,Spain,540
Muslera,Uruguay,570


*5) How many players on a team with a ranking lower than 10 played more than 350 minutes? Return a single number in a column named "superstar".*

**Hint**: To rename a column, use the AS keyword (e.g. `SELECT column_1 AS label`).

In [ ]:
%%sql
select count(surname) as superstar
from PlayersExt
where ranking < 10
  and minutes > 350

 * sqlite://
Done.


superstar
54


*6) What is the average number of passes made by forwards? What about midfielders? Write one query that returns both values with the corresponding position.*

**Hint**: Use the GROUP BY keyword. GROUP BY statements are often used in conjuction with aggregate functions like AVG(), SUM() or COUNT(). 

In [ ]:
%%sql
select position, avg(passes) as "avg_passes"
from Players
where position in ("forward", "midfielder")
group by position

 * sqlite://
Done.


position,avg_passes
forward,50.82517482517483
midfielder,95.2719298245614


### Advanced Queries

Now, on to more challenging questions...

*1) Find all pairs of teams that have the same number of `goalsFor` as well as the same number of `goalsAgainst` as each other. Return the team pairs and their respective numbers of `goalsFor` and `goalsAgainst` (make sure to return each pair only once!).*

**Hint**: You basically need to do a "self join" of the `Teams` table. For that, you need to join different name aliases of the same table. Check [this page](https://www.w3schools.com/sql/sql_join_self.asp) for help.

In [ ]:
%%sql
select *
FROM Teams as T1, Teams as T2
LIMIT 3

 * sqlite://
Done.


team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards,team_1,ranking_1,games_1,wins_1,draws_1,losses_1,goalsFor_1,goalsAgainst_1,yellowCards_1,redCards_1
Brazil,1,5,3,1,1,9,4,7,2,Brazil,1,5,3,1,1,9,4,7,2
Brazil,1,5,3,1,1,9,4,7,2,Spain,2,6,5,0,1,7,2,3,0
Brazil,1,5,3,1,1,9,4,7,2,Portugal,3,4,1,2,1,7,1,8,1


In [ ]:
%%sql
select A.team AS team_A, B.team AS team_B, A.goalsFor, A.goalsAgainst
from Teams A, Teams B
where A.team < B.team 
  and A.goalsFor = B.goalsFor
  and A.goalsAgainst = B.goalsAgainst

 * sqlite://
Done.


team_A,team_B,goalsFor,goalsAgainst
Italy,Mexico,4,5
England,Nigeria,3,5
England,South Africa,3,5
Chile,England,3,5
Chile,Nigeria,3,5
Chile,South Africa,3,5
Cameroon,Greece,2,5
Australia,Denmark,3,6
Nigeria,South Africa,3,5


*2) Find all teams with a ranking below 30 where no player has made more than 150 passes. Return the team's name and ranking.*

**Hint #1**: Consider using the HAVING keyword.

**Hint #2**: You may also want to look up nested queries.

In [ ]:
%%sql
select team, ranking, max(passes) as max_passes
from PlayersExt
group by team
having ranking < 30
  and max_passes <= 150

 * sqlite://
Done.


team,ranking,max_passes
France,9,125
Nigeria,21,111
Switzerland,24,138


*3) Which team has the highest ratio of goalsFor to goalsAgainst?*

In [ ]:
%%sql
select team, goalsFor, goalsAgainst, (1.0 * goalsFor / goalsAgainst) as ratio
from Teams
order by ratio desc
limit 1

 * sqlite://
Done.


team,goalsFor,goalsAgainst,ratio
Portugal,7,1,7.0


*4) Find all teams whose defenders averaged more than 150 passes. Return the team and average number of passes by defenders, in descending order of average passes.*

In [ ]:
%%sql
select team, avg(passes) as avg_passes
from PlayersExt
where position like "defender"
group by team
having avg_passes > 150
order by avg_passes desc

 * sqlite://
Done.


team,avg_passes
Spain,213.0
Brazil,190.0
Germany,189.83333333333334
Netherlands,182.5
Mexico,152.14285714285714


## <font color = 'green' italicized text>Titanic Data (exercises to do at home)</font>

This dataset contains the list of passengers who were on board the Titanic.

Personal information such as their name, gender and age is shown. We can also see information about their journey (which class they were travelling in, how much they paid, etc.), and whether they survived or not.

Feel free to do these exercises on your own time in order to prepare for part 1 of the personal assignment.

In [ ]:
# Load table from CSV file
titanic_url = "https://raw.githubusercontent.com/michalis0/BigScaleAnalytics/master/week2/data/Titanic.csv"
Titanic = pd.read_csv(titanic_url, index_col=0, encoding="utf-8")
%sql drop table if exists Titanic;
%sql persist Titanic

 * sqlite://
Done.
 * sqlite://


'Persisted titanic'

### Preview

In [ ]:
%%sql
select * from Titanic limit 5

 * sqlite://
Done.


last,first,gender,age,class,fare,embarked,survived
Braund,Mr. Owen Harris,M,22.0,3,7.25,Southampton,no
Cumings,Mrs. John Bradley (Florence Briggs Thayer),F,38.0,1,71.2833,Cherbourg,yes
Heikkinen,Miss Laina,F,26.0,3,7.925,Southampton,yes
Futrelle,Mrs. Jacques Heath (Lily May Peel),F,35.0,1,53.1,Southampton,yes
Allen,Mr. William Henry,M,35.0,3,8.05,Southampton,no


### Basic Queries

*1) How many married women over age 50 embarked in Cherbourg?*

**Hint**: You will need to use wildcards.

In [ ]:
%%sql
select count(*) as count
from Titanic
where gender like "F"
  and embarked like "Cherbourg"
  and age > 5
  and first like "%(%)%"

 * sqlite://
Done.


count
24


*2) List the average fare paid by passengers in each of the embarkation cities (along with the city), in descending order of average fare.*

In [ ]:
%%sql
select embarked, avg(fare) as avg_fare
from Titanic
group by embarked
order by avg_fare desc

 * sqlite://
Done.


embarked,avg_fare
Cherbourg,59.95414404761905
Southampton,27.243651393188795
Queenstown,13.276029870129872


*3) What is the most common last name among passengers?*

In [ ]:
%%sql
select last, count(last) as occ
from Titanic
group by last
order by occ desc
limit 1

 * sqlite://
Done.


last,occ
Andersson,9


*4) Write 3 queries:*

* *Total number of passengers*
* *Number of passengers under 30*
* *Number of passengers 30 or older*

*Why do the second and third numbers not add up to the first?*

In [ ]:
%%sql
select count(*) from Titanic

 * sqlite://
Done.


count(*)
891


In [ ]:
%%sql
select count(*) from Titanic where age < 30

 * sqlite://
Done.


count(*)
384


In [ ]:
%%sql
select count(*) from Titanic where age >= 30

 * sqlite://
Done.


count(*)
330


Blanks in SQL tables are given a special value `null`, and conditions like `A is null` and `A is not null` can be used in WHERE statements to check whether attribute A contains blank values or not. 

*5) How many passengers don't have a value for age? Now do your numbers add up?*

In [ ]:
%%sql
select count(*) from Titanic where age is null

 * sqlite://
Done.


count(*)
177


*6) How many passengers were in each of the following categories, and what was their average fare paid?*

* *Male survivors*
* *Female survivors*
* *Male non-survivors*
* *Female non-survivors*

In [ ]:
%%sql
select gender, survived, count(*) as number, avg(fare) as avg_fare
from Titanic
group by gender, survived

 * sqlite://
Done.


gender,survived,number,avg_fare
F,no,81,23.02438518518519
F,yes,233,51.93857339055791
M,no,468,21.960992948717944
M,yes,109,40.82148440366974


### Advanced Queries

*1) Are there any pairs of passengers with the same last name where one is in first class and the other is in third class?*

*If so, return the last name and each person's first name. Label the first name column as "first" for the passenger in first class, and "third" for the passenger in third class.*

In [ ]:
%%sql
select t1.last, t1.first as first, t2.first as third
from Titanic t1, Titanic t2
where t1.last = t2.last
  and t1.first <> t2.first
  and t1.class = 1
  and t2. class = 3

 * sqlite://
Done.


last,first,third
Allen,Miss Elisabeth Walton,Mr. William Henry
Smith,Mr. James Clinch,Mr. Thomas
Smith,Mr. Richard William,Mr. Thomas
Williams,Mr. Charles Duane,"Mr. Howard Hugh ""Harry"""
Flynn,"Mr. John Irwin (""Irving"")",Mr. James
Daly,Mr. Peter Denis,Mr. Eugene Patrick
Williams,Mr. Charles Duane,Mr. Leslie
Carlsson,Mr. Frans Olof,Mr. August Sigfrid
Flynn,"Mr. John Irwin (""Irving"")",Mr. John


*2) Which embarkation cities have more than 40 passengers whose age is missing?*

In [ ]:
%%sql
select embarked, count(*) as missing_age
from Titanic
where age is null
group by embarked
having missing_age > 40

 * sqlite://
Done.


embarked,missing_age
Queenstown,49
Southampton,90


*3) Find all classes where the average fare paid by passengers in that class was more than twice the overall average or less than half the overall average.*

In [ ]:
%%sql
select class, avg(fare) as avg_fare
from Titanic
group by class
having avg_fare > (2 * (select avg(fare) from Titanic))
  or avg_fare < ((select avg(fare) from Titanic) / 2)

 * sqlite://
Done.


class,avg_fare
1,84.15468749999992
3,13.675550101832997


*4) EXTRA DIFFICULT CHALLENGE: List each class and its survival rate, i.e., the fraction of passengers in that class who survived.*

In [ ]:
%%sql
select t1.class, (1.0 * survivors / total_class) as survival_rate
from (select class, count(*) as total_class from Titanic group by class) t1
join (select class, count(*) as survivors from Titanic where survived like 'yes' group by class) t2
on t1.class = t2.class

 * sqlite://
Done.


class,survival_rate
1,0.6296296296296297
2,0.47282608695652173
3,0.24236252545824846


### Titanic Data Modification

Here again, we are giving you the solution for the first two questions to get you acquainted with the modification syntax.

Note: You may want to reload the CSV frequently to reset the data as you experiment with modifications.

In [ ]:
# Load table from CSV file
titanic_url = "https://raw.githubusercontent.com/michalis0/BigScaleAnalytics/master/week2/data/Titanic.csv"
Titanic = pd.read_csv(titanic_url, index_col=0, encoding="utf-8")
%sql drop table if exists Titanic;
%sql persist Titanic

 * sqlite://
Done.
 * sqlite://


'Persisted titanic'

*1) Subtract 5 from the fare paid by all passengers under the age of 10. Then compute the new average fare similar to question 2 in the previous section.*

**Hint #1**: You can put two SQL statements in one cell separated by a semicolon.

**Hint #2**: Use the UPDATE and SET keywords to modify a column (see [documentation](https://www.w3schools.com/sql/sql_update.asp)).

In [ ]:
%%sql
update Titanic
set fare = fare - 5
where age < 10
;
select embarked, avg(fare) as avg_fare
from Titanic
group by embarked
order by avg_fare desc

 * sqlite://
62 rows affected.
Done.


embarked,avg_fare
Cherbourg,59.68628690476191
Southampton,26.86439442724453
Queenstown,13.016289610389611


*2) Create a new table called "Survivors" containing the first and last names of all passengers who survived. Then count the number of tuples in the new table.*

**Hint**: Use the CREATE TABLE keyword.

In [ ]:
%%sql
drop table if exists Survivors
;
create table Survivors as
select first, last
from Titanic 
where survived = "yes"
;
select count(*) as count from Survivors

 * sqlite://
Done.
Done.
Done.


count
342


*3) In the Titanic table, delete all but the passengers who paid more than 300. Then count the number of tuples in the table.*

**Hint**: Use the DELETE FROM keyword.

In [ ]:
%%sql
delete from Titanic
where fare <= 300
;
select count(*) as count from Titanic

 * sqlite://
888 rows affected.
Done.


count
3


*4) In what's left of the table after question 3, insert a new tuple for yourself. You can decide your class, fare, where you embarked, and whether you survived. Then show the whole table.*

**Hint**: Use the INSERT INTO keyword.

In [ ]:
%%sql
insert into Titanic
values ("Tsague", "Estelle", "F", 22, 2, 15, "Cherbourg", "yes")
;
select * from Titanic
limit 5

 * sqlite://
1 rows affected.
Done.


last,first,gender,age,class,fare,embarked,survived
Ward,Miss Anna,F,35.0,1,512.3292,Cherbourg,yes
Cardeza,Mr. Thomas Drake Martinez,M,36.0,1,512.3292,Cherbourg,yes
Lesurer,Mr. Gustave J,M,35.0,1,512.3292,Cherbourg,yes
Tsague,Estelle,F,22.0,2,15.0,Cherbourg,yes
